In [2]:
from itertools import combinations

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import swifter


# Подготовка датасета для загрузки в сеть

In [3]:
base_df = pd.read_csv('data/clean_frame.csv', index_col='ind')
base_df.columns

Index(['ID', 'Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'Пособие',
       'Общежитие', 'Наличие_Матери', 'Наличие_Отца', 'Опекунство', 'Село',
       'Иностранец', 'КодФакультета', 'СрБаллАттестата', 'Статус', 'male',
       'female', 'sex_summ', 'birth_year_int', 'basis', 'language', 'country',
       'region', 'city', 'parents_country'],
      dtype='object')

### Проверка и удаление отрицательных значений

In [4]:
base_df.isna().any().all()

False

In [5]:
base_df = base_df.fillna(0)

In [6]:
def get_accuracy(frame_to_train):
    train, test = train_test_split(frame_to_train, test_size=0.20)
    
    train_input = train.drop('Статус', axis=1)    
    train_output = train['Статус']

    test_input = test.drop('Статус', axis=1)
    test_output = test['Статус']

    clf = RandomForestClassifier(random_state=0)
    clf.fit(train_input, train_output)
    pred = clf.predict(test_input)
    return f1_score(test_output, pred, average='macro', zero_division = 0)

In [7]:
all_columns_list = base_df.columns.to_list()
all_columns_list.remove('ID')
all_columns_list.remove('Статус')
all_columns_list

['Код_группы',
 'Год_Поступления',
 'Год_Окончания_УЗ',
 'Пособие',
 'Общежитие',
 'Наличие_Матери',
 'Наличие_Отца',
 'Опекунство',
 'Село',
 'Иностранец',
 'КодФакультета',
 'СрБаллАттестата',
 'male',
 'female',
 'sex_summ',
 'birth_year_int',
 'basis',
 'language',
 'country',
 'region',
 'city',
 'parents_country']

In [ ]:
bad_columns = []

In [88]:
elements_num = len(all_columns_list)
elements_num = 2

In [89]:
all_combinations = []
list_sizes = 0
min_col = 2
for i in range(min_col, elements_num+1):
    curent_list = list(combinations(all_columns_list, i))
    list_sizes += len(curent_list)
    all_combinations = all_combinations + curent_list
comb_size = len(all_combinations)
comb_size

231

In [90]:
all_combinations

[('Код_группы', 'Год_Поступления'),
 ('Код_группы', 'Год_Окончания_УЗ'),
 ('Код_группы', 'Пособие'),
 ('Код_группы', 'Общежитие'),
 ('Код_группы', 'Наличие_Матери'),
 ('Код_группы', 'Наличие_Отца'),
 ('Код_группы', 'Опекунство'),
 ('Код_группы', 'Село'),
 ('Код_группы', 'Иностранец'),
 ('Код_группы', 'КодФакультета'),
 ('Код_группы', 'СрБаллАттестата'),
 ('Код_группы', 'male'),
 ('Код_группы', 'female'),
 ('Код_группы', 'sex_summ'),
 ('Код_группы', 'birth_year_int'),
 ('Код_группы', 'basis'),
 ('Код_группы', 'language'),
 ('Код_группы', 'country'),
 ('Код_группы', 'region'),
 ('Код_группы', 'city'),
 ('Код_группы', 'parents_country'),
 ('Год_Поступления', 'Год_Окончания_УЗ'),
 ('Год_Поступления', 'Пособие'),
 ('Год_Поступления', 'Общежитие'),
 ('Год_Поступления', 'Наличие_Матери'),
 ('Год_Поступления', 'Наличие_Отца'),
 ('Год_Поступления', 'Опекунство'),
 ('Год_Поступления', 'Село'),
 ('Год_Поступления', 'Иностранец'),
 ('Год_Поступления', 'КодФакультета'),
 ('Год_Поступления', 'СрБалл

In [91]:
combinations_frame = pd.DataFrame(all_combinations, columns=range(elements_num))
combinations_frame


,0,1
0,Код_группы,Год_Поступления
1,Код_группы,Год_Окончания_УЗ
2,Код_группы,Пособие
3,Код_группы,Общежитие
4,Код_группы,Наличие_Матери
...,...,...
226,country,city
227,country,parents_country
228,region,city
229,region,parents_country


In [92]:
def itter_from_frame(row):
    frame_columns = row.dropna().to_list() + ['Статус']
    frame_to_train = base_df.loc[:, frame_columns]
    return get_accuracy(frame_to_train)

In [93]:
start, stop = 0, comb_size
combinations_frame.loc[start:stop, 'accuracy'] = combinations_frame.iloc[start:stop, :].apply(itter_from_frame, axis=1)

In [94]:
combinations_frame.head(30)

,0,1,accuracy
0,Код_группы,Год_Поступления,0.718183
1,Код_группы,Год_Окончания_УЗ,0.716459
2,Код_группы,Пособие,0.751500
3,Код_группы,Общежитие,0.749646
4,Код_группы,Наличие_Матери,0.760877
5,Код_группы,Наличие_Отца,0.754564
6,Код_группы,Опекунство,0.763686
7,Код_группы,Село,0.733674
8,Код_группы,Иностранец,0.741365
9,Код_группы,КодФакультета,0.754032


In [95]:
combinations_frame['accuracy'].describe()

count    231.000000
mean       0.356692
std        0.128148
min        0.247861
25%        0.282115
50%        0.331690
75%        0.363967
max        0.763686
Name: accuracy, dtype: float64

In [96]:
rt = combinations_frame.sort_values('accuracy', ascending=False)
rt.to_csv(f'data/combinations/columns_from_{min_col}_to_{elements_num}.csv')


In [97]:
rt.groupby('accuracy').describe()

0                                 1                            
         count unique             top freq count unique             top freq
accuracy                                                                    
0.247861     1      1          female    1     1      1        language    1
0.248441     2      2  Наличие_Матери    1     2      2    Наличие_Отца    1
0.249309     1      1  Наличие_Матери    1     1      1        sex_summ    1
0.249405     1      1         Пособие    1     1      1          female    1
0.249532     1      1         Пособие    1     1      1      Опекунство    1
...        ...    ...             ...  ...   ...    ...             ...  ...
0.751500     1      1      Код_группы    1     1      1         Пособие    1
0.754032     1      1      Код_группы    1     1      1   КодФакультета    1
0.754564     1      1      Код_группы    1     1      1    Наличие_Отца    1
0.760877     1      1      Код_группы    1     1      1  Наличие_Матери    1
0.763686     1      1      Код_группы    1     1      1      Опекунство    1

[223 rows x 8 columns]

In [ ]:
accuracy = {}
for num, columns in enumerate(all_combinations[:100]):
    frame_columns = list(columns)
    frame_columns.append('Статус')
    frame_to_train = base_df.loc[:, frame_columns]
    accuracy[num] = get_accuracy(frame_to_train)
    print(num)
accuracy


In [ ]:
accuracy

In [ ]:
max_index = max(accuracy, key=accuracy.get)
print(max_index, all_combinations[max_index], accuracy[max_index])